코드설명은   
readme의 노션 링크에서 확인해주세요!

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from konlpy.tag import Okt
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud


url = "https://news.daum.net/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}

html = requests.get(url, headers=headers)
soup = BeautifulSoup(html.content, 'html.parser')
news_block = soup.find(class_="box_g box_news_issue")
news_main = news_block.find_all(class_="tit_g")

news_block_len = len(news_main)

news_content_link = []
news_content_headline = []

for i in range(news_block_len):
    link_element = news_main[i].find('a', class_='link_txt')
    if link_element and 'href' in link_element.attrs:
        news_content_link.append(link_element['href'])
    else:
        news_content_link.append("N/A")  
        
    # 기사 제목을 추출합니다.
    headline_element = news_main[i].find('a')
    if headline_element:
        news_content_headline.append(headline_element.text)
    else:
        news_content_headline.append("N/A")  # 기사 제목을 찾을 수 없는 경우 처리

cleaned_data = []
for item in news_content_headline:
    # 공백 제거 및 제목 부분 추출
    cleaned_item = re.sub(r'\n\s*', '', item.strip())
    cleaned_data.append(cleaned_item)

# 가져온 내용 출력
print(f'가져온 뉴스의 해드라인은 {news_block_len}개 입니다')
for i in range(news_block_len):
    print(f"{i+1} 기사 제목: {cleaned_data[i]}")
    print(f"링크: {news_content_link[i]}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36"}
all_news = []

for news in news_content_link:
    html = requests.get(news, headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    for p in soup.find_all('p', {'dmcf-ptype': 'general'}):
        extracted_text = p.get_text()

        #정규식을 사용하여 특수 문자 및 숫자 제거
        text_data = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣\s]", "", extracted_text)
        #형태소 분석을 위해 Konlpy 라이브러리의 Okt 클래스 사용
        okt = Okt()
        tokens = okt.nouns(text_data)
        #불용어 제외
        stopwords = ["을", "를", "이", "가", "은", "는", "의"]
        filtered_tokens = [word for word in tokens if word not in stopwords]
        print(filtered_tokens)
        #단어 빈도수 카운트
        word_counts = Counter(filtered_tokens)

        wordcloud = WordCloud(font_path="Data preprocessing(2주차))/NanumSquareRoundB.ttf", background_color='white').generate_from_frequencies(word_counts)

        #워드 클라우드 시각화
        plt.figure(figsize=(10, 8))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()







